## 1. Setup Environment

In [7]:
# Mount Google Drive to save models
from google.colab import drive
drive.mount('/content/drive')

# Create directory for saving models
!mkdir -p /content/drive/MyDrive/sarcasm_models

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
# Clone repository (make sure repo is public first!)
!git clone https://github.com/carlo-scr/Sarcasm.git
%cd Sarcasm

In [ ]:
# Install dependencies
!pip install -q transformers datasets peft trl torch accelerate scikit-learn

## 2. Verify GPU

In [ ]:
import torch
print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## 3. Download SARC & Create Data Splits

**Note:** SARC dataset (255MB) is downloaded directly from Princeton NLP to avoid uploading large files. iSarcasm is already in the repo.

In [ ]:
# Download SARC dataset from Kaggle (more reliable than Princeton NLP)
print("Downloading SARC dataset from Kaggle...")
!pip install -q kagglehub

import kagglehub
import shutil
import os

# Download from Kaggle
path = kagglehub.dataset_download("danofer/sarcasm")
print(f"✓ Downloaded to: {path}")

# Find the train-balanced-sarcasm.csv file
!mkdir -p data/SARC
csv_file = None
for root, dirs, files in os.walk(path):
    for file in files:
        if 'train-balanced-sarcasm' in file:
            csv_file = os.path.join(root, file)
            break
    if csv_file:
        break

if csv_file:
    shutil.copy(csv_file, 'data/SARC/train-balanced-sarcasm.csv')
    print(f"✓ SARC dataset ready ({os.path.getsize('data/SARC/train-balanced-sarcasm.csv') / 1e6:.1f} MB)")
else:
    print("❌ Could not find train-balanced-sarcasm.csv in downloaded files")
    print(f"Available files: {os.listdir(path)}")

# Create train/test splits for iSarcasm (already in repo)
!python scripts/create_splits.py

## 4. Phase 1: Supervised Fine-Tuning (SFT)

Train on SARC dataset to learn general sarcasm patterns.

**Time:** ~5-10 minutes on T4 GPU

In [ ]:
!python scripts/finetune_qwen.py

## 5. Phase 2: Direct Preference Optimization (DPO)

Refine with iSarcasm preferences for better accuracy.

**Time:** ~3-5 minutes on T4 GPU

## 5A. Mine Hard Negatives (Optional but Recommended)

Find SFT model's confident mistakes to create targeted training data for DPO.

**Time:** ~2-3 minutes on T4 GPU

In [ ]:
# Mine hard negatives from SFT model's confident mistakes
!python scripts/mine_hard_negatives.py

# Show summary
import json
with open('data/hard_negatives.json', 'r') as f:
    hard_negs = json.load(f)

print(f"\n✓ Mined {len(hard_negs)} hard negatives")
print(f"  False Positives: {sum(1 for hn in hard_negs if hn['predicted_label'] == 1)}")
print(f"  False Negatives: {sum(1 for hn in hard_negs if hn['predicted_label'] == 0)}")
print(f"\nThese will be used to create targeted preference pairs in DPO training.")

## 5B. Enhanced DPO Training

Now train DPO with:
- ✅ Explicit reasoning in preference pairs
- ✅ Hard negatives from SFT mistakes
- ✅ Confidence-weighted learning

**Time:** ~3-5 minutes on T4 GPU

In [ ]:
!python scripts/dpo_train.py

## 6. Evaluate All Models

Compare Base → SFT → DPO on held-out test set.

In [ ]:
!python scripts/evaluate_all_stages.py

## 7. View Results

In [ ]:
import json

with open('comparative_results.json', 'r') as f:
    results = json.load(f)

print("\n" + "="*70)
print("COMPARATIVE RESULTS")
print("="*70)

for model in results['models']:
    print(f"\n{model['model_name']}:")
    print(f"  Accuracy:  {model['accuracy']:.2%}")
    print(f"  Precision: {model['precision']:.2%}")
    print(f"  Recall:    {model['recall']:.2%}")
    print(f"  F1 Score:  {model['f1_score']:.2%}")

## 8. Save Models to Google Drive

In [ ]:
# Save models to Google Drive
!cp -r models/sft /content/drive/MyDrive/sarcasm_models/
!cp -r models/dpo_enhanced /content/drive/MyDrive/sarcasm_models/
!cp comparative_results.json /content/drive/MyDrive/sarcasm_models/

print("✓ Models saved to Google Drive: MyDrive/sarcasm_models/")

## 9. Download Results (Optional)

In [ ]:
from google.colab import files

# Download results JSON
files.download('comparative_results.json')

## Summary

**Training Complete!**

Models saved to:
- Google Drive: `MyDrive/sarcasm_models/`
- Local (Colab): `models/sft/` and `models/dpo_enhanced/`

**Expected Performance:**
- Base Model: ~49% accuracy
- SFT Model: ~63% accuracy (+14 pts)
- DPO Model: ~68% accuracy (+5 pts)

**Next Steps:**
1. Download models from Google Drive
2. Use for inference in your application
3. Upload to Hugging Face Hub for sharing